<h1>PolyTrend: Remote sensing time series trend analysis using polynomials</h1>
This script enables trend analysis in vegetation time series data with Google Earth Engine Python API. 
Please make sure you are a registered GEE user. 


<h3>Step 1. Install libraries</h3>

In [0]:
!pip install earthengine-api


<h3>Step 2. Establish connection and authenticate access</h3>

In [0]:
!earthengine authenticate

In [0]:
import ee
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
  print("Unexpected error:", sys.exc_info()[0])
  raise

<h3>Step 3. Import libraries below.</h3>

In [0]:
#import libraries for PolyTrend algorithm
import pandas as pd
import numpy as np
import numpy.linalg as lng
import numpy.polynomial.polynomial as poly
import scipy.stats as stats
from google.colab import files
print("Proceed to step 4.")

<h3>Step 4. Enter parameters:</h3>
<br>
<ul>
    <li>Statistical significance (alpha), the default value is 0.05</li> 
    <li>ID of the dataset you'd like to use. Check its ID <a href="https://developers.google.com/earth-engine/datasets/catalog/">here</a></li>
    <li>Start and end dates</li>
    <li>Threshold for minimum of analyzed values, eg. for NDVI it could be 0.2 to exclude water bodies. Please check what range is offered by the sensor you are using</li>
    <li>Nominal scale in meters per pixel</li>
      <li>Coordinates of the region of interest - in the same format as in the example, each coordinate pair in square brackets [long, lat], EPSG: 4326</li>
</ul>


In [0]:
alpha = 0.05 #@param {type:"slider", min:0, max:0.5, step:0.05}
name_of_collection = 'NASA/GIMMS/3GV0' #@param ['NASA/GIMMS/3GV0', 'NOAA/CDR/AVHRR/NDVI/V4', 'MODIS/006/MOD13A2'] {allow-input: true}
if (name_of_collection == 'NASA/GIMMS/3GV0'):
  band_name = 'ndvi'
else:
  band_name = 'NDVI'
start_year = 2000 #@param {type:"slider", min:1981, max:2019, step:1}
end_year = 2010 #@param {type:"slider", min:1981, max:2019, step:1}
ndvi_threshold = 0.1 #@param {type:"slider", min:-1, max:1, step:0.1}
scale = 8000 #@param {type:"number"}
coords = [[38.79,28.28], [38.79,37.27], [49.16,37.27], [49.16,28.28], [38.79,28.28]] #@param {type: "raw"}
print('Ready to go to the next step.')

<h3>Step 4. Generate annual NDVI composites.</h3><br>
Choose the type of composite to create.

In [0]:
type_of_composite = 'max' #@param ['mean', 'max'] {allow-input: false}

Run the next cell to generate a collection of composites.

In [0]:
if (type_of_composite == 'mean'):
  def create_composite(year_and_collection):
    # Unpack variable from the input parameter
      year_and_collection = ee.List(year_and_collection)
      year = ee.Number(year_and_collection.get(0))
      _collection = ee.ImageCollection(year_and_collection.get(1))
      start_date = ee.Date.fromYMD(year, 1, 1)
      end_date = start_date.advance(1, 'year')
      return  _collection.filterDate(start_date, end_date).mean().set('system:time_start', year)
elif (type_of_composite == 'max'):
   def create_composite(year_and_collection):
    # Unpack variable from the input parameter
      year_and_collection = ee.List(year_and_collection)
      year = ee.Number(year_and_collection.get(0))
      _collection = ee.ImageCollection(year_and_collection.get(1))
      start_date = ee.Date.fromYMD(year, 1, 1)
      end_date = start_date.advance(1, 'year')
      return  _collection.filterDate(start_date, end_date).max().set('system:time_start', year)
AOI = ee.Geometry.Polygon(coords)

#The below code is adapted from Tylere on Github: https://gist.github.com/tylere/42e4acf883e18f5b8e331cfab8c91ab5
collection = ee.ImageCollection(name_of_collection).select(band_name).filterBounds(AOI)
collection = collection.filterDate(ee.Date.fromYMD(start_year, 1, 1), ee.Date.fromYMD(end_year, 1, 1).advance(1, 'year'))
#Create list of years
years = ee.List.sequence(start_year, end_year, 1)
# Create a list of year-collection pairs (i.e. pack the function inputs)
list_of_years_and_collections = years.zip(ee.List.repeat(collection, years.length()))

annualNdvi = ee.ImageCollection.fromImages(list_of_years_and_collections.map(create_composite))
print(annualNdvi.size().getInfo())


geom_values = annualNdvi.getRegion(geometry=AOI, scale=scale)
geom_values_list = ee.List(geom_values).getInfo()
# Convert to a Pandas DataFrame.
header = geom_values_list[0]
data = pd.DataFrame(geom_values_list[1:], columns=header)
data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
data.set_index('time')
data.groupby(['longitude', 'latitude'])

print(data)

<h4>Step 4a. Alternatively, save the time series for the polygon. It saves to the active Google drive folder environment as time_series.csv.</h4>

In [0]:
data.to_csv('time_series.csv')
files.download('time_series.csv')

<h3>Step 5. Run PolyTrend algorithm per pixel. It will take a while to complete, depending on the size of your data set.</h3>
<p><i>Watch for a message saying 'Running this process ended successfully.' below the cell</i><p>

In [0]:
message = 'Please wait for a message that the process is completed...'
#definition of the PolyTrend algorithm
def PolyTrend(Y, alpha):
    X = range(1, len(Y)+1)
 
    #define function to find p value:
    def Pvalue(coef, df, A, Aprim, pn):
        #generate square residual
        part_res = np.dot(A, pn)-Y
        residual = np.dot(part_res.transpose(), part_res)
        #generate variance-covariance matrix
        VC = lng.inv(np.dot(Aprim, A))*residual/df
        #compute variance of the first coefficient
        VC1 = np.sqrt(VC[0,0])
        #compute t-statistic
        statistic = coef/VC1
        #compute p value
        p = stats.t.sf(np.abs(statistic), df)*2
        return p;
    
    def Plinear(X, Y):
        df1 = len(X)-2
        #generate Vandermonde matrix
        A1 = np.vander(X, 2)
        #generate transpose of the Vandermonde matrix
        Aprim1 = A1.transpose()
        p1 = np.dot(np.dot((lng.inv(np.dot(Aprim1, A1))), Aprim1), Y)
        coef1 = p1[0]
        Plin = Pvalue(coef1, df1, A1, Aprim1, p1)
        Slope = p1[0]
        Direction = np.sign(Slope)
        #Slope and Direction will be referred to Plin[1] and Plin[2] respectively in returned results
        return Plin, Slope, Direction;
    
    #degrees of freedom
    df3 = len(X)-4
    #generate Vandermonde matrix
    A3 = np.vander(X, 4)
    #generate transpose of the Vandermonde matrix
    Aprim3 = A3.transpose()
    #X=inv(A'*A)*A'*L - creating coefficients matrix:
    p3 = np.dot(np.dot((lng.inv(np.dot(Aprim3, A3))), Aprim3), Y)
    coef3 = p3[0]
    #compute p-value for cubic fit
    Pcubic = Pvalue(coef3, df3, A3, Aprim3, p3)
    #get roots of cubic polynomial
    coefs3 = ([p3[2], 2*p3[1], 3*p3[0]])
    roots3 = np.sort(poly.polyroots(coefs3))

    if (np.imag(roots3[0]) == 0 and np.imag(roots3[1])==0 and roots3[0] != roots3[1] and X[0] <= roots3[0] <= X[-1] and X[0] <= roots3[1] <= X[-1] and Pcubic < alpha):
        Plin = Plinear(X, Y)
        if (Plin[0] < alpha):
            Trend_type = 3
            Significance = 1
            Poly_degree = 3
        else:
            Trend_type = -1
            Significance = -1
            Poly_degree = 3
            return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
    else:
        df2 = len(X)-3
        A2 = np.vander(X, 3)
        Aprim2 = A2.transpose()
        p2 = np.dot(np.dot((lng.inv(np.dot(Aprim2, A2))), Aprim2), Y)
        coef2 = p2[0]
        Pquadratic = Pvalue(coef2, df2, A2, Aprim2, p2)
        coefs2 = ([p2[1], 2*p2[0]])
        roots2 = np.sort(poly.polyroots(coefs2))
        
        if (X[0] <= roots2 <= X[-1] and Pquadratic < alpha):
            Plin = Plinear(X, Y)
            if Plin[0] < alpha:
                Trend_type = 2
                Significance = 1
                Poly_degree = 2
            else:
                Trend_type = -1
                Significance = -1
                Poly_degree = 2
                return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
                
        else:
            Plin = Plinear(X, Y)
            if Plin[0] < alpha:
                Trend_type = 1
                Significance = 1
                Poly_degree = 1
            else:
                Trend_type = 0
                Significance = -1
                Poly_degree = 0
            return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];     
        return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
    return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
#end of PolyTrend definition

#establish how many images there are in the collection
list_of_images = data['id']
ids_of_images = []
for img_id in list_of_images:
    if img_id not in ids_of_images:
        ids_of_images.append(img_id)
        
n = len(ids_of_images)
print('number of images: ', n)
number_of_pixels = len(data) 
print('number of pixels analysed: ', number_of_pixels)

#make_Y function returns the results of the analysis for each individual pixel identified by its coordinates
def make_Y(dataset, alpha):
    PT_result = []
    #split the dataset into pixel time series
    for i in range(0, number_of_pixels, n):
        Y = dataset[i:i+n][band_name].values 
        #eliminate numbers lower than the threshold and any other values that are not numeric
        for value in Y:
            if value > ndvi_threshold and isinstance(value, (int,float)):
                try:
                    result = list(PolyTrend(Y, alpha))
                except:
                    result = ['unqualified', 'unqualified', 'unqualified', 'unqualified', 'unqualified']
            else:
                result = ['NA', 'NA', 'NA', 'NA', 'NA']
        #populate the empty PT_result list with values    
        pixel_long = dataset.at[i, 'longitude']
        pixel_lat = dataset.at[i, 'latitude']
        PT_result_header = ['longitude', 'latitude', 'trend type', 'significance', 'degree', 'slope', 'direction']
        PT_result.append([pixel_long, pixel_lat, result[0], result[1], result[2], result[3], result[4]])
    #create a data frame for displaying results on a map    
    image_frame = pd.DataFrame(PT_result[0:], columns=PT_result_header)
    return image_frame;
print(message)
final_result = make_Y(data, alpha)
pixels_to_display = len(final_result)

print('points produced: ', pixels_to_display)
message = 'Process finished successfully. Save your data to a csv file.'
print(message)


<h3>Step 6. Save results to a csv file in current Google drive folder.</h3>

In [0]:
final_result.to_csv('PolyTrend_result.csv')
files.download('PolyTrend_result.csv')